<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=300 height=300 align=left class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Fall 2023<br>
<font color=3C99D size=5>
    Practical Assignment 1 - Search <br>
<font color=696880 size=4>
    Reza Soumi, Rouzbeh Pirayadi, Amir Mohammad Izadi

____

# Personal Data

In [23]:
# Set your student number
student_number = '99100177'
Name = 'Amirsalar'
Last_Name = 'Safaei Ghaderi'

# Rules
- Make sure that all of your cells can be run perfectly. 

## Libraries

In [24]:
import pygame, time, csv
import numpy as np
from time import sleep
import heapq
from IPython.display import HTML
# my requirements
from math import sqrt
from typing import Tuple, Literal, Set

# Uninformed & Informed Search (100 Points)

<font size=4>
Author: Reza Soumi
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

## Problem Definition

In This Problem, We want to investigate Maze Problem with the focus on implementing IDS and A-Star algorithm to solve it. 

The concept of the **Digital Twin** was first introduced in 1960 during NASA's Apollo project. In this project, many simulation engines was created in the ground where sensory data from the spacecraft was sent to simulation devices. The model would then update itself with this data and send response or action commands back to the spacecraft using advanced telecommunication protocols. This concept has recently gained popularity.

If you are interested in learning more about digital twin in the Apollo NASA project, you can read this [link](https://blogs.sw.siemens.com/simcenter/apollo-13-the-first-digital-twin/) which explains how these systems helped astronauts return home safely. Additionally, there is a [TEDx](https://www.ted.com/talks/karen_willcox_how_digital_twins_could_help_us_predict_the_future?language=en) talk dedicated to this concept that you may find interesting.

Now, let's consider a scenario where we have launched a simple spacecraft into space without any IoT devices. Unfortunately, our robot has become stuck in a maze. The starting position is at the top left corner and the destination is at the bottom right corner, as shown in the picture below.

<!-- ![Sample Image](Pics/map.PNG) -->
<div style="text-align:center">
  <img src="Pics/map.PNG" alt="alt text" style="width:50%;"/>
</div>

To solve this problem, we need to implement an intelligent digital twin for the spacecraft. This digital twin will gather information from the spacecraft and determine which action to take - up, left, down, or right. However, we cannot explore the entire map at once and can only take one step forward at a time.

## Code

You can use the following data structures, and feel free to use any other data structures as needed, but these should suffice.

In [25]:
###########################################
###### no need to change any thing ########
###########################################

class Node:
    def __init__(self, pos, parent, cost=None):
        self.x = pos[0]
        self.y = pos[1]
        self.parent = parent
        self.cost = cost

    def position(self):
        return (self.x, self.y)

class Queue:
    def __init__(self):
        self.list = []

    def push(self,item):
        self.list.insert(0,item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0

class PriorityQueue:
    """ O(1) access to the lowest-priority item """
    def  __init__(self):
        self.heap = []
        self.count = 0

    def push(self, item, priority):
        entry = (priority, self.count, item)
        heapq.heappush(self.heap, entry)
        self.count += 1

    def pop(self):
        (_, _, item) = heapq.heappop(self.heap)
        return item

    def isEmpty(self):
        return len(self.heap) == 0

    def update(self, item, priority):
        # If item already in priority queue with higher priority, update its priority and rebuild the heap.
        for index, (p, c, i) in enumerate(self.heap):
            if i == item:
                if p <= priority:
                    break
                del self.heap[index]
                self.heap.append((priority, c, item))
                heapq.heapify(self.heap)
                break
        else:
            self.push(item, priority)

class Stack:
    def __init__(self):
        self.list = []

    def push(self,item):
        self.list.append(item)

    def pop(self):
        return self.list.pop()

    def isEmpty(self):
        return len(self.list) == 0

In [26]:
# I added this for convenience, and so I don't have to repeat myself.
# These are basic requirements for a greed search
class GridSearch:
    _dx = (0, 1, -1, 0)
    _dy = (1, 0, 0, -1)

    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim

    def get_successors(self, x, y):
        return [(x + self._dx[i], y + self._dy[i]) for i in range(4)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

The code below takes a CSV file's path as the map address. In this CSV file, a value of 1 represents a passable block where the robot can navigate, while a value of 0 represents an impassable wall where the robot cannot go. There's no need to be concerned about Digital Twins at this point; we will implement them later.

In [27]:
###########################################
###### no need to change any thing ########
###########################################

def solve_maze(map_address, algorithm):
    grid = np.genfromtxt(map_address, delimiter=',', dtype=np.int32)
    num_rows, num_columns = len(grid), len(grid[0])
    empty_block_count = np.count_nonzero(grid == 1)

    # Define start & goal positions
    start_pos = (0,0)
    goal_pos = (num_rows-1, num_columns-1)

    grid[0, 0] = 2
    grid[-1, -1] = 3

    grid_dim = (num_rows-1, num_columns-1)

    black, white, green, red, grey, blue, magenta = (0,0,0), (255, 255, 255), (50,205,50), (255,99,71), (211,211,211), (153,255,255), (255,0,255)
    idx_to_color = [black, white, green, red, blue, magenta]

    height = 15
    width = height
    margin = 1

    pygame.init()

    WINDOW_SIZE = [660, 660]
    screen = pygame.display.set_mode(WINDOW_SIZE)

    pygame.display.set_caption(f"{algorithm} Pathfinder. Solving: {map_address}")

    done = False
    run = False
    close = False

    clock = pygame.time.Clock()

    digital_twin = None

    if algorithm == "BFS":
        digital_twin = BFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "DFS":
        digital_twin = DFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "O_DFS":
        digital_twin = Optimized_DFS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "IDS":
        digital_twin = IDS_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    elif algorithm == "A_Star":
        digital_twin = A_Star_Digital_Twin(start_pos=start_pos, goal_pos=goal_pos, grid_dim=grid_dim)
    else:
        return None
    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True

            elif event.type == pygame.KEYDOWN:
                run = True
                start_t0 = time.time()

        screen.fill(grey)

        for row in range(num_rows):
            for column in range(num_columns):
                color = idx_to_color[grid[row, column]]
                pygame.draw.rect(screen, color, [(margin + width) * column + margin, (margin + height) * row + margin, width, height])

        clock.tick(60)
        pygame.display.flip()

        if run == True:
            sleep(0.01)
            solution, done, grid = digital_twin.update(grid=grid)

        if done == True:
            print(f"Total empty block numbers: {empty_block_count}")
            print(f"Explored block numbers: {np.count_nonzero(grid == 4)}")
            for pos in solution:
                grid[pos[0], pos[1]] = 5

            screen.fill(grey)

            for row in range(num_rows):
                for column in range(num_columns):
                    color = idx_to_color[grid[row, column]]
                    pygame.draw.rect(screen, color, [(margin + width) * column + margin, (margin + height) * row + margin, width, height])

            clock.tick(60)
            pygame.display.flip()

    print(f"Your maze solved with {algorithm} algorithm.")
    print(f"--- finished {time.time()-start_t0:.3f} s---")
    while not close:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                close = True

            elif event.type == pygame.KEYDOWN:
                close = True
    pygame.quit()

Below is an implementation of our digital twin using the Depth-First Search (DFS) algorithm. Take a moment to explore how it functions.

In [4]:
###########################################
###### no need to change any thing ########
###########################################

class DFS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.stack = Stack()
        self.stack.push(Node(pos=start_pos, parent=None))

    def get_successors(self, x, y):
        return [(x+1, y), (x-1, y), (x, y+1), (x, y-1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        curr_state = self.stack.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]: # 1: empty cell has not explored yet, 3: goal cell
                self.stack.push(Node(pos=step, parent=curr_state))

                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break

            grid[x, y] = 4 # visited

        return solution_path, done, grid

You can run the code below to see how our exploration works (it is recommended to run it on your local machine instead of Colab).

In [6]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="DFS")

Total empty block numbers: 881
Explored block numbers: 868
Your maze solved with DFS algorithm.
--- finished 12.847 s---


In [21]:
class Optimized_DFS_Digital_Twin:
    def __init__(self, start_pos, goal_pos, grid_dim):
        self.start_pos = start_pos
        self.goal_pos = goal_pos
        self.grid_dim = grid_dim
        self.stack = Stack()
        self.stack.push(Node(pos=start_pos, parent=None))

    def get_successors(self, x, y):
        return [(x-1, y), (x, y-1), (x+1, y), (x, y+1)]

    def is_valid_cell(self, pos):
        return 0 <= pos[0] <= self.grid_dim[0] and 0 <= pos[1] <= self.grid_dim[1]

    def backtrack_solution(self, curr_node):
        return self._backtrack(curr_node)

    def _backtrack(self, curr_node):
        return [] if curr_node.parent is None else self._backtrack(curr_node.parent) + [curr_node.position()]

    def update(self, grid):
        curr_state = self.stack.pop()
        x, y = curr_state.position()
        done = False
        solution_path = []

        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]: # 1: empty cell has not explored yet, 3: goal cell
                self.stack.push(Node(pos=step, parent=curr_state))

                if step == self.goal_pos:
                    done = True
                    solution_path = self.backtrack_solution(curr_state)
                    break

            grid[x, y] = 4 # visited

        return solution_path, done, grid

In [22]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="O_DFS")

Total empty block numbers: 881
Explored block numbers: 304
Your maze solved with O_DFS algorithm.
--- finished 4.954 s---


After running the code above and entering it on the screen, you should see the result, which resembles the GIF below.

In [7]:
gif_path = 'Pics/Animation.gif'
html_code = f'<div style="display: flex; justify-content: center;"> <img src="{gif_path}" alt="Your GIF" style="width: 40%;"/></div>'
HTML(html_code)

As you can see, all of the blocks have been explored using DFS algorithm. So, we need a better algorithm for this maze problem. You can think and devise a solution for this problem at this stage before proceeding.

Let's investigate other uninformed search methods. Your task now is to implement Breadth-First Search (BFS) algorithm to solve this maze problem. (Hint: You can simply just change the data structures used in the code provided above to convert DFS to BFS)

In [11]:
########################################################
######### implement BFS algorithm (10 Points) ##########
########################################################

class BFS_Digital_Twin(GridSearch):


    def __init__(self, start_pos, goal_pos, grid_dim):
        super().__init__(start_pos, goal_pos, grid_dim)
        self.queue = Queue()
        self.queue.push(Node(pos=start_pos, parent=None))

    def update(self, grid):
        """
        Input: grid (2D array)
        Output:
            solution_path (List of tuples, empty if no solution found)
            done (Boolean, True if the goal is reached, False otherwise)
            grid (Updated 2D array)
        """
        done = False
        solution_path = []
        curr_state = self.queue.pop()
        x, y = curr_state.position()
        for step in self.get_successors(x, y):
            if self.is_valid_cell(step) and grid[step[0], step[1]] in [1, 3]:
                self.queue.push(Node(pos=step, parent=curr_state))
                if step == self.goal_pos:
                  done = True
                  solution_path = self.backtrack_solution(curr_state)
                  break
        grid[x, y] = 4
        return solution_path, done, grid

You can run the code below to see how BFS algorithm works for solving this maze problem.

In [12]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="BFS")

Total empty block numbers: 881
Explored block numbers: 358
Your maze solved with BFS algorithm.
--- finished 5.818 s---


It seems that BFS algorithm provides a good solution for our maze problem. Now, let's try implementing the Iterative Deepening Search (IDS) algorithm in the code below. The depth must increase incrementally (one per request to the digital twin) according to the game policy.

In [30]:
########################################################
######### implement IDS algorithm (15 Points) ##########
########################################################

class IDS_Digital_Twin(GridSearch):
    def __init__(self, start_pos, goal_pos, grid_dim):
        super().__init__(start_pos, goal_pos, grid_dim)
        self.max_depth = 0
        self.stack = Stack()
        self.stack.push((0, Node(pos=self.start_pos, parent=None)))

    def update(self, grid):
        next_stack = Stack()
        done = False
        solution_path = []
        while not self.stack.isEmpty() and not done:
            depth, cur_node = self.stack.pop()
            x, y = cur_node.position()
            grid[x, y] = 4
            if depth == self.max_depth:
                next_stack.push((depth, cur_node))
                continue
            for next_node_pos in self.get_successors(x,y):
                if self.is_valid_cell(next_node_pos) and grid[next_node_pos[0], next_node_pos[1]] in (1, 3):
                    self.stack.push((depth + 1, Node(pos=next_node_pos, parent=cur_node)))

                    if next_node_pos == self.goal_pos:
                        done = True
                        solution_path = self.backtrack_solution(cur_node)
                        break

        self.max_depth += 1
        self.stack = next_stack

        return solution_path, done, grid


In [31]:
solve_maze(map_address="mazes/maze_0.csv", algorithm="IDS")

Total empty block numbers: 881
Explored block numbers: 360
Your maze solved with IDS algorithm.
--- finished 4.749 s---


Question: 

So far, we have explored three uninformed search methods: DFS, BFS, and IDS. Briefly explain the advantages and disadvantages of each method in the context of the Maze Problem and compare their performance based on your observations (consider both time and memory). Suppose there is two destination points, address specific points based on this scenario.

Your Answer: (10 Points)
Time Complexity:
BFS has an advantage is mazes that have a lot of dead ends specially with longer distance than the goal itself it can solve the maze visiting only state with distance of at most d(d is the distance from start to goal) but we can design a maze in which dfs and bfs algorithm perform the same. IDS can have a lower time complexity than dfs in cases which there are a lot of distant dead ends, but it is always more time-consuming than BFS because for each distance it restarts. IDS time complexity is O(z*d^2) = O(z + 2z + ... + dz) and BFS time complexity is O(dz)(z is branching factor and d is distance from start to goal)
Memory Complexity:
BFS and DFS(graph dfs) both need O(n^2) memory to hold which state has been seen, we can't eliminate this in bfs but we can use tree dfs that might have a larger time complexity and fail when there is no goal but it solves the problem in O(max distance)
but IDS can always solve memory in O(d * z) d is distance from start to goal and z is branching factor. but as d and z increase d * z converges to n^2

2 destination points scenario:
if we have to reach both of them d in the complexity formulas becomes max(distance(start, first-destination), distance(start, second-destination)) and if reaching one of them suffices, then d in the complexity formulas becomes min(distance(start, first-destination), distance(start, second-destination)).



Question:

Provide a simple solution to improve the DFS algorithm's performance in terms of explored block numbers, Based on the game information.

Your Answer: (5 Points)
when visiting a cell neighbours, first visit (x+1, y) and (x, y+1) (in the scenario that the goal in the bottom right of the grid. In general case visit the neighbours that are in the goals direction we can figure this out in O(1) using simple if cases. this can cause dfs to not explore some unnecessary blocks in wide areas that are connected, in narow spaces with lots of dead ends this wont help us much. you can see the resualt in O_DFS code(optimal dfs).

Now, let's turn our attention to a second maze map, 'maze_1.csv'. solve this maze using the BFS algorithm.

In [16]:
solve_maze(map_address="mazes/maze_1.csv", algorithm="BFS")

Total empty block numbers: 881
Explored block numbers: 799
Your maze solved with BFS algorithm.
--- finished 13.323 s---


As you can see, BFS may not always perform optimally, as it can end up exploring a majority of the blocks. Let's proceed by implementing the A-Star algorithm in the next cell.

In [28]:
########################################################
######### implement A* algorithm (50 Points) ###########
########################################################
class A_Star_Digital_Twin(GridSearch):
    def __init__(self, start_pos, goal_pos, grid_dim, heuristic_mode: Literal["manhattan", "euclidean"] = "manhattan"):
        super().__init__(start_pos, goal_pos, grid_dim)

        self.heuristic_mode= heuristic_mode
        self.pqueue = PriorityQueue()
        self.seen: Set[Tuple[int, int]] = set()

        self._push_to_pqueue(Node(pos=start_pos, parent=None), 0)

    def _heuristic(self, pos):
        if self.heuristic_mode == "manhattan":
            return abs(pos[0] - self.goal_pos[0]) + abs(pos[1] - self.goal_pos[1])
        if self.heuristic_mode == "euclidean":
            return sqrt(pow(pos[0] - self.goal_pos[0],2) + pow(pos[1] - self.goal_pos[1],2))

    def _push_to_pqueue(self, node: Node, cost_g: int):
        cost_h = self._heuristic(node.position())
        self.pqueue.update((cost_g, cost_h, node), cost_h + cost_g)

    def update(self, grid):
        done = False
        solution_path = []

        cost_g, cost_h, cur_node = self.pqueue.pop()
        x, y = cur_node.position()
        grid[x, y] = 4
        self.seen.add(cur_node.position())

        for next_node_pos in self.get_successors(x,y):
            if next_node_pos in self.seen or not (self.is_valid_cell(next_node_pos) and grid[next_node_pos[0], next_node_pos[1]] in [1, 3]):
                continue
            self._push_to_pqueue(node=Node(pos=next_node_pos, parent=cur_node), cost_g=cost_g+1)
            if next_node_pos == self.goal_pos:
                done = True
                solution_path = self.backtrack_solution(cur_node)
                break

        return solution_path, done, grid

Now solve the 'maze_1.csv' with your A-star algorithm.

In [29]:
solve_maze(map_address="mazes/maze_1.csv", algorithm="A_Star")

Total empty block numbers: 881
Explored block numbers: 612
Your maze solved with A_Star algorithm.
--- finished 9.914 s---


Question:

Explain your heuristic and observation's result. Also, compare the A* method with Uniformed search in the context of the Maze problem.

Answer: 
I used manhattan distance for the heuristic function(The manhattan distance from goal).Because we move to non-diagonal neighbours (4 neighbours) the monotonicity of H(n) is trivial because we move like manhattan and every move we make if its in the direction of goal the manhattan distance lowers by one and if its not it grows by 1. we know that cost of every move is considered to be so h(n) <= h(n+1) + cost holds and because H(goal) = 0 we showed in practical questions that if a heuristic function is consistent and H(goal) = 0 we can show that its admissible as well.

using uniformed search explores more dead ends and unnecessary blocks because it can reach them with less cost but this lower cost aggregated together can make algorithm pay more cost in total which is less than ideal A* tries to fix this by trying to guess the cost left from next state to goal and doesn't search state which don't need a large cost to get to them but may need large cost reaching the goal.